In [1]:
import sqlite3 as lite
import pandas as pd

def get_old_data():
    con = lite.connect('batch.db')
    curr=con.cursor()
    curr.execute("SELECT * FROM bat")
    data=curr.fetchall()
    curr.close()
    return data


df=pd.DataFrame(get_old_data(),columns=['id','LotNo','RollNo','Party', 'Variety','GrossWt', 'TareWt','CoreWt', 'NetWt','Date','Status'])


In [2]:
variety=df["Variety"].drop_duplicates()
rolls=[]
for data in variety:
    result = df[df['Variety'] == str(data)]['RollNo'].max()
    rolls.append(result)

variety_roll = pd.DataFrame({
    'Variety_id': range(1, len(variety) + 1),  # Auto-incrementing id starting from 1
    'Variety': variety,
    'Last_RollNo': rolls
})
variety_roll.head()

sqlite_db = 'batch_new.db'

# Create a connection to SQLite database
conn = lite.connect(sqlite_db)

# Save DataFrame to SQLite database
variety_roll.to_sql(name='Variety', con=conn, if_exists='replace', index=False)

# Commit changes and close connection
conn.commit()
conn.close()

In [3]:
variety_id_map = dict(zip(variety_roll['Variety'], variety_roll['Variety_id']))

df["Variety"]=df["Variety"].map(variety_id_map)


In [4]:
df

,id,LotNo,RollNo,Party,Variety,GrossWt,TareWt,CoreWt,NetWt,Date,Status
0,836,1,155,PARLE PRODUCTS,1,37.080,0.08,1.15,35.850,01-07-2024,Done
1,837,1,156,PARLE PRODUCTS,1,31.020,0.08,1.15,29.790,01-07-2024,Done
2,838,1,157,PARLE PRODUCTS,1,31.670,0.08,1.15,30.440,01-07-2024,Done
3,839,1,158,PARLE PRODUCTS,1,32.770,0.08,1.15,31.540,01-07-2024,Done
4,840,1,159,PARLE PRODUCTS,1,32.740,0.08,1.15,31.510,01-07-2024,Done
...,...,...,...,...,...,...,...,...,...,...,...
3410,5766,1,340,PARLE PRODUCTS,1,28.290,00.065,0.7,27.520,01-07-2024,Pending
3411,5767,1,341,PARLE PRODUCTS,1,28.830,00.065,0.7,28.060,01-07-2024,Pending
3412,5768,1,342,PARLE PRODUCTS,1,35.340,00.065,0.7,34.580,01-07-2024,Pending
3413,5769,1,343,PARLE PRODUCTS,1,36.120,00.065,0.7,35.350,01-07-2024,Pending


In [5]:
df.rename(columns={'Variety': 'Variety_id'}, inplace=True)

sqlite_db = 'batch_new.db'


conn = lite.connect(sqlite_db)


df.to_sql(name='bat', con=conn, if_exists='replace', index=False,)

# Commit changes and close connection
conn.commit()
conn.close()

In [6]:

con = lite.connect('batch_new.db')
curr=con.cursor()
curr.execute("SELECT * FROM bat INNER JOIN Variety ON Variety.Variety_id=bat.Variety_id")
data=curr.fetchall()
curr.close()


In [7]:
new_df = pd.DataFrame(data,columns=['id','LotNo','RollNo','Party', 'Variety_id','GrossWt', 'TareWt','CoreWt', 'NetWt','Date','Status','Variety_id','Variety','Last_RollNo'])

In [8]:
new_df[["id","LotNo","RollNo","Party","Variety","GrossWt","TareWt","CoreWt","NetWt","Date","Status"]]

,id,LotNo,RollNo,Party,Variety,GrossWt,TareWt,CoreWt,NetWt,Date,Status
0,836,1,155,PARLE PRODUCTS,570MM HM SHEET BLUE 4010539,37.080,0.08,1.15,35.850,01-07-2024,Done
1,837,1,156,PARLE PRODUCTS,570MM HM SHEET BLUE 4010539,31.020,0.08,1.15,29.790,01-07-2024,Done
2,838,1,157,PARLE PRODUCTS,570MM HM SHEET BLUE 4010539,31.670,0.08,1.15,30.440,01-07-2024,Done
3,839,1,158,PARLE PRODUCTS,570MM HM SHEET BLUE 4010539,32.770,0.08,1.15,31.540,01-07-2024,Done
4,840,1,159,PARLE PRODUCTS,570MM HM SHEET BLUE 4010539,32.740,0.08,1.15,31.510,01-07-2024,Done
...,...,...,...,...,...,...,...,...,...,...,...
3410,5766,1,340,PARLE PRODUCTS,570MM HM SHEET BLUE 4010539,28.290,00.065,0.7,27.520,01-07-2024,Pending
3411,5767,1,341,PARLE PRODUCTS,570MM HM SHEET BLUE 4010539,28.830,00.065,0.7,28.060,01-07-2024,Pending
3412,5768,1,342,PARLE PRODUCTS,570MM HM SHEET BLUE 4010539,35.340,00.065,0.7,34.580,01-07-2024,Pending
3413,5769,1,343,PARLE PRODUCTS,570MM HM SHEET BLUE 4010539,36.120,00.065,0.7,35.350,01-07-2024,Pending


In [10]:
def GetAll():
    con=lite.connect("batch_new.db")
    cur=con.cursor()
    cur.execute("SELECT LotNo,RollNo,Party,Variety,GrossWt,TareWt,CoreWt,NetWt,Date,Status FROM bat INNER JOIN Variety ON Variety.Variety_id=bat.Variety_id")
    rows=cur.fetchall()
    con.close()
    return rows
print(GetAll())

[('1', '155', 'PARLE PRODUCTS', '570MM HM SHEET BLUE 4010539', '37.080', '0.08', '1.15', '35.850', '01-07-2024', 'Done'), ('1', '156', 'PARLE PRODUCTS', '570MM HM SHEET BLUE 4010539', '31.020', '0.08', '1.15', '29.790', '01-07-2024', 'Done'), ('1', '157', 'PARLE PRODUCTS', '570MM HM SHEET BLUE 4010539', '31.670', '0.08', '1.15', '30.440', '01-07-2024', 'Done'), ('1', '158', 'PARLE PRODUCTS', '570MM HM SHEET BLUE 4010539', '32.770', '0.08', '1.15', '31.540', '01-07-2024', 'Done'), ('1', '159', 'PARLE PRODUCTS', '570MM HM SHEET BLUE 4010539', '32.740', '0.08', '1.15', '31.510', '01-07-2024', 'Done'), ('1', '160', 'PARLE PRODUCTS', '570MM HM SHEET BLUE 4010539', '28.480', '0.08', '1.15', '27.250', '01-07-2024', 'Done'), ('1', '29', 'PARLE PRODUCTS', '470MM HM SHEET 4008115', '27.050', '0.08', '0.9', '26.070', '01-07-2024', 'Done'), ('1', '30', 'PARLE PRODUCTS', '470MM HM SHEET 4008115', '26.210', '0.08', '0.9', '25.230', '01-07-2024', 'Done'), ('1', '31', 'PARLE PRODUCTS', '470MM HM SHEET